In [18]:
uv --version
gdown --help

uv 0.7.13


usage: gdown [-h] [-V] [-O OUTPUT] [-q] [--fuzzy] [--id] [--proxy PROXY]
             [--speed SPEED] [--no-cookies] [--no-check-certificate]
             [--continue] [--folder] [--remaining-ok] [--format FORMAT]
             [--user-agent USER_AGENT]
             url_or_id

positional arguments:
  url_or_id             url or file/folder id (with --id) to download from

optional arguments:
  -h, --help            show this help message and exit
  -V, --version         display version (default: None)
  -O OUTPUT, --output OUTPUT
                        output file name/path; end with "/"to create a new
                        directory (default: None)
  -q, --quiet           suppress logging except errors (default: False)
  --fuzzy               (file only) extract Google Drive's file ID (default:
                        False)
  --id                  [DEPRECATED] flag to specify file/folder id instead of
                        url (default: False)
  --proxy PROXY         <protocol:/

In [ ]:
uv run gdown "https://drive.google.com/uc?id=1Q0zg9zqTaZUvt42uxzE_0gcfnFvjwi33" -O gnomix/pretrained_gnomix_models.tar.gz

Preprocess the GVCF into a format that is compatible with WhatsHap

In [ ]:
bcftools norm --threads 24 -m -any -f reference/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta reference/EG100001.merged.g.vcf.gz | \
  bcftools view --threads 24 -v snps -m2 -M2 -Oz -o reference/EG100001.snps.vcf.gz

bcftools index --threads 24 -t reference/EG100001.snps.vcf.gz

Lines   total/split/joined/realigned/mismatch_removed/dup_removed/skipped:	337580708/5583944/0/200294/0/0/0


Might need to filter the bam for only primary alignments

Below filters for:
- not primary alignment (0x100)
- supplementary alignment (0x800)

In [ ]:
whatshap phase -o output/EG100001.phased.vcf.gz --reference=reference/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta reference/EG100001.snps.vcf.gz reference/EG100001.bam > WhatsHap.log 2>&1

Liftover to hg19

In [10]:
docker run -it --rm -v "$(pwd)":/data -v /tmp:/tmp -w /data clarity001/bcftools-liftover:latest \
    /bin/bash -c "bcftools +liftover -Ou output/EG100001.phased.vcf.gz -- -s reference/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta -f reference/hg19.fa -c reference/hg38ToHg19.over.chain.gz | bcftools sort -Oz -o output/EG100001.phased.hg19.vcf.gz -W=tbi"

Writing to /tmp/bcftools.2IuQyX
INFO/AS_InbreedingCoeff is handled by AGR rule
INFO/AS_QD is handled by AGR rule
INFO/MLEAC is handled by AGR rule
INFO/MLEAF is handled by AGR rule
FORMAT/GP is handled by AGR rule
FORMAT/PG is handled by AGR rule
FORMAT/PL is handled by AGR rule
Lines   total/swapped/reference added/rejected:	4450819/25773/772/208367
Merging 2 temporary files
Done
Cleaning


Test gnomix

In [12]:
uv run gnomix/gnomix.py --help

...
--------------------------------------------------------------------------------
-----------------------------------  Gnomix  -----------------------------------
--------------------------------------------------------------------------------
When using this software, please cite: 
Helgi Hilmarsson, Arvind S Kumar, Richa Rastogi, Carlos D Bustamante, 
Daniel Mas Montserrat, Alexander G Ioannidis: 
"High Resolution Ancestry Deconvolution for Next Generation Genomic Data" 
https://www.biorxiv.org/content/10.1101/2021.09.19.460980v1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Error: Incorrect number of arguments.
Usage when training a model from scratch:
   $ python3 gnomix.py <query_file> <output_basename> <chr_nr> <phase> <genetic_map_file> <reference_file> <sample_map_file>
Usage when u

In [13]:
cp gnomix/config.yaml .

'gnomix/config.yaml' -> './config.yaml'


In [ ]:
mkdir -p output/snps_only
uv run gnomix/gnomix.py output/EG100001.phased.hg19.vcf.gz output/snps_only chr22 True gnomix/pretrained_gnomix_models/chr22/model_chm_22.pkl > output/snps_only/gnomix.log 2>&1

...
--------------------------------------------------------------------------------
-----------------------------------  Gnomix  -----------------------------------
--------------------------------------------------------------------------------
When using this software, please cite: 
Helgi Hilmarsson, Arvind S Kumar, Richa Rastogi, Carlos D Bustamante, 
Daniel Mas Montserrat, Alexander G Ioannidis: 
"High Resolution Ancestry Deconvolution for Next Generation Genomic Data" 
https://www.biorxiv.org/content/10.1101/2021.09.19.460980v1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Launching in pre-trained mode...
Loading model...
Launching inference...
Loading and processing query file...
- Number of SNPs from model: 317408
- Number of SNPs from file: 62013
- Number of intersecting SNPs: 46033


Alternatively if you want to keep all variants, just normalize

In [17]:
bcftools norm --threads 32 -m -any -f reference/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta -o reference/EG100001.normalized.vcf.gz reference/EG100001.merged.g.vcf.gz

bcftools index --threads 24 -t reference/EG100001.normalized.vcf.gz

Lines   total/split/joined/realigned/mismatch_removed/dup_removed/skipped:	337580708/5583944/0/200294/0/0/0


Filter bam for only primary alignments

In [19]:
samtools view -@ 64 -b -F 0x900 reference/EG100001.bam -o reference/EG100001.primary.bam

samtools index -@ 64 reference/EG100001.primary.bam

Phase the normalized gVCF file and use only primary alignments

In [20]:
whatshap phase -o output/EG100001.phased.primaryonly.vcf.gz --reference=reference/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta reference/EG100001.normalized.vcf.gz reference/EG100001.primary.bam > WhatsHap.log 2>&1

Liftover phased VCF to hg19

In [21]:
docker run -it --rm -v "$(pwd)":/data -v /tmp:/tmp -w /data clarity001/bcftools-liftover:latest \
    /bin/bash -c "bcftools +liftover -Ou output/EG100001.phased.primaryonly.vcf.gz -- -s reference/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta -f reference/hg19.fa -c reference/hg38ToHg19.over.chain.gz | bcftools sort -Oz -o output/EG100001.phased.primaryonly.hg19.vcf.gz -W=tbi"

Writing to /tmp/bcftools.3mK7Zz
INFO/AS_InbreedingCoeff is handled by AGR rule
INFO/AS_QD is handled by AGR rule
INFO/MLEAC is handled by AGR rule
INFO/MLEAF is handled by AGR rule
FORMAT/GP is handled by AGR rule
FORMAT/PG is handled by AGR rule
FORMAT/PL is handled by AGR rule
Lines   total/swapped/reference added/rejected:	343586717/30993/7014/2756224
Merging 19 temporary files
Done
Cleaning


Run the ancestry deconvolution tool (gnomix)

In [22]:
mkdir -p output/normalized_primary_only
uv run gnomix/gnomix.py output/EG100001.phased.primaryonly.hg19.vcf.gz output/normalized_primary_only chr22 True gnomix/pretrained_gnomix_models/chr22/model_chm_22.pkl > output/normalized_primary_only/gnomix.log 2>&1

...
--------------------------------------------------------------------------------
-----------------------------------  Gnomix  -----------------------------------
--------------------------------------------------------------------------------
When using this software, please cite: 
Helgi Hilmarsson, Arvind S Kumar, Richa Rastogi, Carlos D Bustamante, 
Daniel Mas Montserrat, Alexander G Ioannidis: 
"High Resolution Ancestry Deconvolution for Next Generation Genomic Data" 
https://www.biorxiv.org/content/10.1101/2021.09.19.460980v1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Launching in pre-trained mode...
Loading model...
Launching inference...
Loading and processing query file...
- Number of SNPs from model: 317408
- Number of SNPs from file: 4652795
- Number of intersecting SNPs: 8568